In [1]:
from google.colab import drive
drive.mount('/content/drive')
base_path = '/content/drive/MyDrive/NLP/'
# base_path = ''

Mounted at /content/drive


In [2]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [3]:
import json
import pandas as pd
import torch
import numpy as np
from transformers import (
    BertTokenizer,
    BertPreTrainedModel,
    BertModel,
    TrainingArguments,
    Trainer,
    EvalPrediction
)
from transformers import EarlyStoppingCallback
from datasets import Dataset as HFDataset
from sklearn.metrics import accuracy_score, f1_score
import torch.nn as nn

class REClassifier(BertPreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        self.bert = BertModel(config)
        self.classifier = nn.Linear(config.hidden_size, config.num_labels)
        self.init_weights()

    def forward(self, input_ids, attention_mask, labels=None, **kwargs):
        outputs = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask
        )
        pooled_output = outputs.pooler_output
        logits = self.classifier(pooled_output)

        loss = None
        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(logits.view(-1, self.config.num_labels), labels.view(-1))

        return {"loss": loss, "logits": logits}

def load_and_preprocess_data(file_path):
    data_list = []
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            data = json.loads(line.strip())
            case = data

            patient = []
            other_entity = []

            for i in case['spans']:
                if i['label'] == 'PATIENT':
                    patient.append(i)
                else:
                    other_entity.append(i)

            for i in patient:
                entity_list = other_entity.copy()
                for j in case['relations']:
                    if i['token_end'] == j['head']:
                        for k in entity_list:
                            if k['start'] == j['child_span']['start']:
                                entity_list.remove(k)
                        temp = {
                            'text': case['text'],
                            'patient': case['text'][j['head_span']['start']:j['head_span']['end']],
                            'entity': case['text'][j['child_span']['start']:j['child_span']['end']],
                            'entity_type': j['label'],
                            'label': 1
                        }
                        data_list.append(temp)

                for k in entity_list:
                    temp = {
                        'text': case['text'],
                        'patient': i['text'],
                        'entity': k['text'],
                        'entity_type': k['label'],
                        'label': 0
                    }
                    data_list.append(temp)

    return pd.DataFrame(data_list)

def preprocess_function(examples, tokenizer, max_len=256):
    input_texts = [
        f"[PATIENT] {p} [ENTITY] {e} [SEP] {t}"
        for p, e, t in zip(examples['patient'], examples['entity'], examples['text'])
    ]

    encodings = tokenizer(
        input_texts,
        truncation=True,
        max_length=max_len,
        padding="max_length",
        return_tensors="pt"
    )

    return {
        "input_ids": encodings["input_ids"],
        "attention_mask": encodings["attention_mask"],
        "labels": examples['label']
    }

def compute_metrics(p: EvalPrediction):
    preds = np.argmax(p.predictions, axis=1)
    labels = p.label_ids
    acc = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds, average="weighted")
    return {"accuracy": acc, "f1": f1}

In [4]:
df_train = load_and_preprocess_data(base_path + "dataset/cleaned/RE/RE_train_data.jsonl")
df_test = load_and_preprocess_data(base_path + "dataset/cleaned/RE/RE_test_data.jsonl")

model_name = "prajjwal1/bert-tiny"
# model_name = "allenai/scibert_scivocab_uncased"
# model_name = "dmis-lab/biobert-base-cased-v1.1"
# model_name = "microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract"

tokenizer = BertTokenizer.from_pretrained(model_name)

temp_hf_dataset = HFDataset.from_pandas(df_train)
test_hf_dataset = HFDataset.from_pandas(df_test)

temp_tokenized_dataset = temp_hf_dataset.map(
    lambda examples: preprocess_function(examples, tokenizer),
    batched=True,
    remove_columns=temp_hf_dataset.column_names
)

test_dataset = test_hf_dataset.map(
    lambda examples: preprocess_function(examples, tokenizer),
    batched=True,
    remove_columns=test_hf_dataset.column_names
)

dataset_split = temp_tokenized_dataset.train_test_split(test_size=0.2, seed=42)
train_dataset = dataset_split["train"]
eval_dataset = dataset_split["test"]

model = REClassifier.from_pretrained(
    model_name,
    num_labels=2,
    ignore_mismatched_sizes=True
)

training_args = TrainingArguments(
    output_dir=base_path+"./model/RE",
    num_train_epochs=5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    learning_rate=5e-4,
    # weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs",
    logging_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,
    report_to="none",
    seed=42
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
    # callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
)

trainer.train()

eval_results = trainer.evaluate()
print(f"Final assessment result: {eval_results}")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.txt: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/285 [00:00<?, ?B/s]

Map:   0%|          | 0/69 [00:00<?, ? examples/s]

Map:   0%|          | 0/155 [00:00<?, ? examples/s]

pytorch_model.bin:   0%|          | 0.00/17.8M [00:00<?, ?B/s]

Some weights of REClassifier were not initialized from the model checkpoint at prajjwal1/bert-tiny and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipython-input-1075951783.py:54: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


model.safetensors:   0%|          | 0.00/17.7M [00:00<?, ?B/s]

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.637500,0.645931,0.642857,0.503106
2,0.562100,0.591144,0.642857,0.503106
3,0.512700,0.530778,0.642857,0.503106
4,0.479800,0.489231,0.642857,0.503106
5,0.462200,0.472073,0.642857,0.503106


Final assessment result: {'eval_loss': 0.6459313631057739, 'eval_accuracy': 0.6428571428571429, 'eval_f1': 0.5031055900621119, 'eval_runtime': 0.0141, 'eval_samples_per_second': 993.692, 'eval_steps_per_second': 70.978, 'epoch': 5.0}


In [5]:
from sklearn.metrics import confusion_matrix,accuracy_score,precision_score,recall_score,f1_score
test_predictions = trainer.predict(test_dataset)
pred_logits = test_predictions.predictions  # (n_samples, n_classes)
pred_labels = np.argmax(pred_logits, axis=1)
test_results = trainer.evaluate(eval_dataset=test_dataset)
precision = precision_score(test_dataset["labels"], pred_labels)
recall = recall_score(test_dataset["labels"], pred_labels)

print(f"Evaluation results of the test set: {test_results}")

print(f"Accuracy: {test_results['eval_accuracy']:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {test_results['eval_f1']:.4f}")

Evaluation results of the test set: {'eval_loss': 0.5330928564071655, 'eval_accuracy': 0.7870967741935484, 'eval_f1': 0.6933271223943169, 'eval_runtime': 0.1074, 'eval_samples_per_second': 1443.418, 'eval_steps_per_second': 46.562, 'epoch': 5.0}
Accuracy: 0.7871
Precision: 0.7871
Recall: 1.0000
F1-Score: 0.6933


In [6]:
import pandas as pd
df_all = load_and_preprocess_data(base_path + "dataset/cleaned/RE/RE_all_data.jsonl")

all_hf_dataset = HFDataset.from_pandas(df_all)

all_dataset = all_hf_dataset.map(
    lambda examples: preprocess_function(examples, tokenizer),
    batched=True,
    remove_columns=temp_hf_dataset.column_names
)

all_predictions = trainer.predict(all_dataset)
all_pred_logits = all_predictions.predictions
all_pred_labels = np.argmax(all_pred_logits, axis=1)

res = {
    'Patient': all_hf_dataset['patient'],
    'Entity': all_hf_dataset['entity'],
    'Label': all_pred_labels
}
res_df = pd.DataFrame(res)
print(res_df[:50])
print(res_df[50:100])
print(res_df[100:150])
print(res_df[150:200])
print(res_df[200:250])

Map:   0%|          | 0/224 [00:00<?, ? examples/s]

       Patient                                             Entity  Label
0    Patient 1                                                 27      1
1    Patient 1                                          childhood      1
2    Patient 1                                 migraine headaches      1
3    Patient 1                                           anxiety.      1
4    Patient 1                                                 33      1
5   Patient 2)                                                 33      1
6   Patient 2)                                          childhood      1
7   Patient 2)                                 migraine headaches      1
8   Patient 2)                                           anxiety.      1
9   Patient 2)                                                 27      1
10   Patient 1                              dystonic toe curling,      1
11   Patient 1                                      action tremor      1
12   Patient 1                                  red